In [3]:
import pandas as pd
import numpy as np 
import json

In [4]:
data = pd.read_csv("who_suicide_statistics.csv")

In [5]:
df = pd.DataFrame(data)
countrys = pd.unique(df[df["year"] == 2013]["country"])
df_wout_nan = df.dropna()
df_wout_nan

,country,year,sex,age,suicides_no,population
24,Albania,1987,female,15-24 years,14.0,289700.0
25,Albania,1987,female,25-34 years,4.0,257200.0
26,Albania,1987,female,35-54 years,6.0,278800.0
27,Albania,1987,female,5-14 years,0.0,311000.0
28,Albania,1987,female,55-74 years,0.0,144600.0
...,...,...,...,...,...,...
43759,Virgin Islands (USA),2015,male,25-34 years,2.0,4609.0
43760,Virgin Islands (USA),2015,male,35-54 years,1.0,12516.0
43761,Virgin Islands (USA),2015,male,5-14 years,0.0,7291.0
43762,Virgin Islands (USA),2015,male,55-74 years,0.0,12615.0


In [6]:
countrys_avg = dict()
for country in countrys:
    countrys_avg[country] = df_wout_nan[df_wout_nan["country"] == country].groupby(["year"])["suicides_no"].sum().mean()



In [21]:
rangos_etarios = ['5-14 years', '15-24 years', '25-34 years', '35-54 years',
       '55-74 years', '75+ years']

In [7]:
rangos_etarios = df["age"].unique()
sexos = df["sex"].unique()
df[df["country"] == "Chile"].groupby(["age", "sex"])["suicides_no"].sum()['15-24 years']["female"]

1776.0

In [24]:

datos = []
df = df_wout_nan
# Resumen de suicidios 2015
for country in countrys:
    country_summary = {}
    country_summary["pais"] = country
    value = df[(df["country"] == country) & (df["year"] == 2013)]["suicides_no"].sum() / df[(df["country"] == country) & (df["year"] == 2013)]["population"].sum()
    if not np.isnan(value):
        country_summary["summary"] =  (value*100000) #Numero de suicidios por cada 100.000 Habitantes
    else:
        continue
    
    country_summary["rango_etario"] = []
    rango_etario = df[(df["country"] == country) & (df["year"] == 2013)].groupby(["age", "sex"])["suicides_no"].sum()
    for rango in rangos_etarios:
        dicto = {"rango": rango, "values": []}
        for sex in sexos:
            dicto["values"].append({"sex": sex,"rate" : rango_etario[rango][sex]})
        country_summary["rango_etario"].append(dicto)
    
    country_summary["years"] = []
    # Para cada año vemos si hubieron suicidios y los agregamos al diccionario
    for year in range(1987, 2014):
        year_sum = df[(df["country"] == country) & (df["year"] == year)]["suicides_no"].sum()
        if year_sum.any():
            country_summary["years"].append({"year" :str(year), "suicides": year_sum})
        else:
            country_summary["years"].append({"year" : str(year), "suicides": int(countrys_avg[country])})
    datos.append(country_summary)


In [25]:
with open('data.json', 'w') as outfile:
    json.dump(datos, outfile, indent=4)